Let's use `pip` to install some libraries that we are going to use

* pandas: Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, built on top of the Python programming language.
* seaborn: Seaborn is a Python data visualization library based on matplotlib.
* matplotlib: Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python. 
* numpy: Numpy is the core library for scientific computing in Python. It provides a high-performance multidimensional array object, and tools for working with these arrays.
* dvc: Data Version Control for your machine learning projects. 
* s3fs: 3FS builds on botocore to provide a convenient Python filesystem interface for S3.

In [ ]:
!pip install -U pandas seaborn matplotlib numpy dvc s3fs --quiet

Restart the Jupyter kernel after installing pip packages.

Let's import pandas using the alias `pd`

In [1]:
import pandas as pd
import numpy as np

Let's read the CSV from a S3 bucket.

The [datasets](https://www.kaggle.com/mlg-ulb/creditcardfraud) contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. 

* Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset.
* The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 
* Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

In [3]:
import dvc.api
import s3fs
import os

resource_url = dvc.api.get_url(
    path='creditcard.csv',
    repo=os.environ['DATA_REPO'],
    rev='v1.0')

print("Data is from this S3 bucket:\n{}".format(resource_url))

# Pandas doens't support endpoint_url
# https://github.com/pandas-dev/pandas/pull/29050
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': os.environ['S3_ENDPOINT_URL']})
df = pd.read_csv(fs.open(resource_url))

Data is from this S3 bucket:
s3://mlflow-obc-c23f8109-fff0-4cbc-ae94-53bb7390287e/dvcf4g2/e9/0efcb83d69faf99fcab8b0255024de


In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Let's look at the data types. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
# Descritive stats 
df.describe().round(decimals=2)

In [ ]:
# Check for null and missing values
data = df.fillna(np.nan)
data.isnull().sum()

In [ ]:
df['Amount'].describe().round(decimals=2)

In [ ]:
fraud = df[df['Class']==1]
normal = df[df['Class']==0]

In [ ]:
fraud.shape

In [ ]:
normal.shape

There are 492 fraud and 284315 normal transactions.

In [ ]:
# Let's look at the precentage of fraud vs normal
print('Normal', round(fraud.shape[0]/len(df) * 100,2), '% of the dataset')
print('Frauds', round(normal.shape[0]/len(df) * 100,2), '% of the dataset')

In [ ]:
normal.Amount.describe().round(decimals=2)

In [ ]:
fraud.Amount.describe().round(decimals=2)

In [ ]:
# What is the average fraud value?
print('Average Fraud: {} | Average Normal: {}'.format(normal['Amount'].mean() , fraud['Amount'].mean()))

In [ ]:
# What is the highest fraud value?
print('Max Fraud: {} | Max Normal: {}'.format(normal['Amount'].max() , fraud['Amount'].max()))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set()

In [ ]:
sns.countplot('Class', data=df)
plt.title('Class Distributions \n (0: No Fraud || 1: Fraud)')

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

sns.distplot(fraud.Amount, bins=50, kde=False, ax=ax1, axlabel=False)
sns.distplot(normal.Amount, bins=50, kde=False, ax=ax2, axlabel=False)

f.suptitle('Transaction by Class')

ax1.set_title('Fraud')
ax2.set_title('Normal')
plt.xlabel('Amount ($)')
plt.ylabel('Number of Transactions')
plt.yscale('log')

In [ ]:
# Calculate pearson correlation
plt.figure(figsize=(20,15))
plt.title('Correlation matrix', fontsize=16)
sns.heatmap(df.corr(method='pearson'), annot=True, fmt=".2f", cmap="coolwarm")

'Class' is less correlated with ‘Amount’ and ‘Time’ which suggests it is hard to predict whether transaction is fraudulent or not from ‘Amount’ and ‘Time’ details of transaction.

'Class' is negatively correlated with ‘V3’, ‘V7’, ‘V10’, ‘V12’, ‘V14’, ‘V17’ and positively correlated with ‘V2’, ‘V4’, ‘V11’. The other correlations are relatively small.